# Connect to SAP HANA

...the usual way like you've done so far 

In [ ]:
%run "../01-check_setup.ipynb"

# Query HANA table with `REAL_VECTOR` to find two similar pets

In [ ]:
source_table="IMAGES"
source_schema="VECTORS"

## Closest neighbours (for image embeddings)

Use the Vector Engine's [cosine similarity calculation for two vectors](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-vector-engine-guide/cosine-similarity-063e1366a7d54735b98b2513ea4a88c9) to find a pet similar to a randomly chosen one. 

Try it a few times to get some different results, but as well check the similarity score returned for these two images.

In [ ]:
import random
random_record = random.randint(1, myconn.table(table=source_table, schema=source_schema).count())
print(f"Random record number: {random_record}")

In [ ]:

sql = f'''
SELECT 
  "B"."NAME" AS "BREED", 
  "A"."NAME" AS "SIMILAR_ONE", 
  "COSINE_SIMILARITY"("A"."IV", "B"."IV") AS "SIMILARITY_SCORE",
  "B"."IMAGE_NAME", 
  "A"."IMAGE_NAME" AS "SIMILAR_IMAGE_NAME",
  "B"."IMAGE", 
  "A"."IMAGE" AS "SIMILAR_IMAGE"
FROM 
  "{source_schema}"."{source_table}" AS "A"
INNER JOIN 
  (SELECT 
     ROW_NUMBER() OVER () AS "row_num", *
   FROM "{source_schema}"."{source_table}"
  ) AS "B" 
ON "A"."NAME" <> "B"."NAME"
WHERE "B"."row_num" = {random_record}
ORDER BY "SIMILARITY_SCORE" DESC;
'''

hdf = myconn.sql(sql)
#print(hdf.select_statement)
df=hdf.head(3).collect()

display(df.iloc[:,:3].style.hide(axis='index'))


Retrieve Base64-encoded images of thoese two similar pets and display them side-by-side. 

In [ ]:
from IPython.display import display, HTML

In [ ]:
# Create HTML for side-by-side display
html_code = f"""
<div style="display: flex; justify-content: space-around;">
    <div style="text-align: center;">
        <img src="data:image/webp;base64,{df.IMAGE.iloc[0]}" width="400"><br>{df.BREED.iloc[0]}
    </div>
    <div style="text-align: center;">
        <img src="data:image/webp;base64,{df.SIMILAR_IMAGE.iloc[0]}" width="400"><br>{df.SIMILAR_ONE.iloc[0]}
    </div>
</div>
"""

# Display the HTML
display(HTML(html_code))


## Not challenged enough?

Here are the two additional ideas for you:

1. Find which pet is the least similar to any other pet (where its top similarity score is the lowest).
2. Got a pet? Or got a picture of some other cat or dog? Find what the most similar pet in the table "IMAGES" is.